In [1]:
import pandas as pd
import numpy as np
import feather

df = (feather.read_dataframe('/home/SHARED/SOLAR/data/oahu_min.feather')
             .set_index('Datetime'))

In [66]:
info = pd.read_csv('/home/SHARED/SOLAR/data/info.csv')

In [67]:
info['Location'] = info['Location'].str.replace('(HL)?_', '')

In [70]:
info.set_index('Location', inplace=True)

In [72]:
info

,Pakbus,Latitude,Longitude,Sensor,CF
Location,,,,,
DH3,201,21.31236,-158.08463,PY66499,141.76
DH4,202,21.31303,-158.08505,PY66500,118.01
DH5,203,21.31357,-158.08424,PY66501,104.46
DH10,204,21.31183,-158.08554,PY66502,109.06
DH11,205,21.31042,-158.08530,PY66503,114.71
DH9,206,21.31268,-158.08688,PY66504,141.92
DH2,207,21.31451,-158.08534,PY66505,106.39
DH1,208,21.31533,-158.08700,PY66519,107.34
AP6,209,21.30812,-158.07935,PY66521,117.34


In [48]:
df1 = df.stack().reset_index().rename(columns={'level_1': 'Radiation_Location', 0: 'Values'})

In [49]:
df1[['Radiation', 'Location']] = df1['Radiation_Location'].str.split('_', n=1, expand=True)

In [52]:
df2 = df1.pivot_table(index=['Datetime', 'Location'], columns='Radiation', values='Values').reset_index()

In [57]:
df2['Datetime'] = df2['Datetime'].dt.tz_localize('HST')

In [69]:
df2.head()

Radiation,Datetime,Location,GH,GT
0,2010-03-19 14:15:00-10:00,AP1,476.328,NaN
1,2010-03-19 14:15:00-10:00,AP3,382.777,NaN
2,2010-03-19 14:15:00-10:00,AP4,351.610,NaN
3,2010-03-19 14:15:00-10:00,AP5,390.092,NaN
4,2010-03-19 14:15:00-10:00,AP6,353.928,343.313


In [79]:
import pysolar

In [81]:
def clear_sky(s, info=None):
    lat, lon = info.loc[s['Location'], ['Latitude', 'Longitude']]
    deg = pysolar.solar.get_altitude(lat, lon, s['Datetime'])
    if deg <= 0:
        radiation = 0.
    else:
        radiation = pysolar.radiation.get_radiation_direct(s['Datetime'], deg)
    return(radiation)

In [ ]:
df2['ClearSky'] = df2[['Datetime', 'Location']].apply(clear_sky, axis=1, info=info)

In [ ]:
# put negative GH to 0 and normalize with ClearSky